In [ ]:
import torch

from lib.models.temporal_predictor import TemporalPredictor


def load_resnet18_temporal_model(checkpoint):
    model = TemporalPredictor("resnet18",
                              dim=512,
                              hidden_dim=128)
    data = torch.load(checkpoint, map_location="cpu")

    model.load_state_dict(data["model_dict"])
    #model = model.backbone
    print(f"Loaded ResNet18 model from {checkpoint}")
    print(f"{sum(p.numel() for p in model.parameters()):,} parameters")
    model.eval()

    return model

model = load_resnet18_temporal_model("models/temporal/temporal_18_beta_detach/checkpoint_90636.pth")

model = model.to("cuda:0")

In [9]:
from pyphoon2.DigitalTyphoonDataset import DigitalTyphoonDataset as DTD
from torchvision import transforms as T
from lib.utils.fisheye import FishEye

transforms = T.Compose([
    T.ToTensor(),
    FishEye(256, 0.2),
])

def transform_func(obj):
    img, labels = obj
    img_range = [150, 350]
    img, labels = obj
    img = (img - img_range[0])/(img_range[1]-img_range[0])

    return transforms(img.astype(np.float32)), labels


dataset = DTD(image_dir="/fs9/gaspar/data/WP/image/",
              metadata_dir="/fs9/gaspar/data/WP/metadata/",
              metadata_json="/fs9/gaspar/data/WP/metadata.json",
              get_images_by_sequence=True,
              labels=("year", "month", "day", "hour", "grade"),#("grade", "pressure", "wind"),
              split_dataset_by="sequence",
              load_data_into_memory='track',
              filter_func= lambda x: x.grade() > 2 and x.grade() < 6,
              transform=None,
              ignore_list=[],
              verbose=False)

In [ ]:
import numpy as np
from torchvision import transforms as T
from lib.utils.fisheye import FishEye
from lib.utils.dataset import PreprocessedTyphoonDataset

transforms = T.Compose([
    #T.ToTensor(),
    FishEye(256,0.2),
])

def transform_func(obj):
    img_range = [150, 350]
    seq= obj
    seq = (seq - img_range[0])/(img_range[1]-img_range[0])
    seq = torch.from_numpy(seq.astype(np.float32))
    seq = seq.unsqueeze(1)
    return transforms(seq)#, labels


dataset = PreprocessedTyphoonDataset(image_dir="/fs9/gaspar/data/WP/image/",
              metadata_dir="/fs9/gaspar/data/WP/metadata/",
              metadata_json="/fs9/gaspar/data/WP/metadata.json",
              labels=("grade", "pressure", "wind"),
              split_dataset_by="sequence",
              load_data_into_memory='track',
              filter_func= lambda x: x.grade() < 6 and x.year() > 1990,
              transform=None,
              ignore_list=[],
              verbose=False,
              feature_extractor=model
              )

In [21]:
transforms = T.Compose([
    T.ToTensor(),
    FishEye(256,0.2),
])

def transform_func(img):
    img_range = [150, 350]
    img = (img - img_range[0])/(img_range[1]-img_range[0])
    print(img.shape)
    return transforms(img.astype(np.float32))

seq, labels = dataset[540]
print(seq.shape)

idx1 = 32
idx2 = 34
device = "cuda:0"
img1, img2 = transform_func(seq[idx1]).to(device), transform_func(seq[idx2]).to(device)

with torch.no_grad():
    print(labels[idx1])
    print(labels[idx2])
    print(img2.shape)
    print(f"Prediction: {model(img1.unsqueeze(0), img2.unsqueeze(0))}")

(129, 512, 512)
(512, 512)
(512, 512)
[2000    7    4   16    5]
[2000    7    4   18    5]
torch.Size([1, 256, 256])
Prediction: tensor([[0.7838]], device='cuda:0')


In [1]:
from lib.utils.dataset import SequenceTyphoonDataset as STD
from torch.utils.data import DataLoader
import numpy as np
import random
import torch

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

dataset = STD(labels=["month", "day", "hour", "pressure", "wind"],
              preprocessed_path="resnet18_imagenet",
              x=[0,1,2,3,4],
              y=[3,4],
              num_inputs=12,
              num_preds=1,
              interval=3,
              filter_func= lambda x: x.grade() < 6,
              output_all=True)
train, val, test = dataset.random_split([0.7, 0.15, 0.15], split_by="sequence")

print(f"\n{len(train)} train sequences")
print(f"{len(val)} val sequences")
print(f"{len(test)} test sequences")

test_loader = DataLoader(test,
                        batch_size=1,
                        shuffle=False,
                        num_workers=0)


[735, 157, 157] 1049
1049

735 train sequences
157 val sequences
157 test sequences


In [3]:
from lib.models.lstm_predictor import LSTM, AttentionLSTM

def _load_checkpoint(model, path):
    data = torch.load(path)
    model.load_state_dict(data["model_dict"])

    print("="*100)
    print(f"Loading model from checkpoint {path}")
    print("="*100)

    return model

model = LSTM(
    69+512,
    hidden_size=1024,
    num_layers=2,
    output_size=2
)

model = _load_checkpoint(model, "/fs9/gaspar/forecoon/models/ts/larger_imagenet/checkpoint_1200.pth")



Loading model from checkpoint /fs9/gaspar/forecoon/models/ts/larger_imagenet/checkpoint_1200.pth


In [4]:
loader=iter(test_loader)


In [5]:
labels = next(loader)[0]
print(labels.shape)


torch.Size([107, 581])


In [6]:
from lib.utils.dataset import NORMALIZATION

pressure = lambda x: x*NORMALIZATION['pressure'][1]+NORMALIZATION['pressure'][0]
wind = lambda x: x*NORMALIZATION['wind'][1]+NORMALIZATION['wind'][0]

def evaluate_results(ys, preds, verbose=False):
    ys = np.array([pressure(ys[0]), wind(ys[1])])
    preds = np.array([pressure(preds[0]), wind(preds[1])])
    diff = np.abs(preds-ys)
    #diff_rel = diff/ys

    if verbose:
        print(f"Pressure| true: {ys[0]:.3f} pred: {preds[0]:.3f} | diff: {diff[0]:.3f}")# ({diff_rel[0]*100:.3f}%)")
        print(f"Wind|\t  true: {ys[1]:.3f} pred: {preds[1]:.3f} | diff: {diff[1]:.3f}")# ({diff_rel[1]*100:.3f}%)")

    return diff#, diff_rel


In [7]:

def retrieve_labels(vec):
    print(f"Pressure: {vec[0]*NORMALIZATION['pressure'][1]+NORMALIZATION['pressure'][0]}")
    print(f"Wind: {vec[1]*NORMALIZATION['wind'][1]+NORMALIZATION['wind'][0]}")
    #print(f"Grade: {torch.argmax(vec[2:])}")


start = 6
ins = labels[test_loader.dataset.dataset.slice_inputs(start)].unsqueeze(0)
outs = labels[test_loader.dataset.dataset.slice_outputs(start), test_loader.dataset.dataset.y]
with torch.no_grad():
    evaluate_results(outs.cpu().squeeze(), model(ins).squeeze().cpu(), True)

Pressure| true: 1008.000 pred: 1010.517 | diff: 2.517
Wind|	  true: 0.000 pred: -1.577 | diff: 1.577


In [ ]:
from tqdm import tqdm

errors = []

for seq in tqdm(test_loader):
    labels = seq[0]
    for start in range(len(labels) - 13):
        ins = labels[test_loader.dataset.dataset.slice_inputs(start), test_loader.dataset.dataset.x].unsqueeze(0)
        outs = labels[test_loader.dataset.dataset.slice_outputs(start), test_loader.dataset.dataset.y]
        with torch.no_grad():
            errors.append(evaluate_results(outs.squeeze(), model(ins).squeeze()))

pressure, wind = zip(*errors)
pressure = np.mean(pressure)
wind = np.mean(wind)
print(pressure, wind)

In [8]:
from copy import deepcopy

@torch.no_grad()
def forecast(model, labels, start, num_preds):
    ws = test_loader.dataset.dataset.num_inputs
    labs = labels[test_loader.dataset.dataset.slice_inputs(start)]
    errors = []
    for i in range(num_preds):
        #print(labs.shape)
        ins = labs[-ws:]
        outs = labels[test_loader.dataset.dataset.slice_outputs(start+i), test_loader.dataset.dataset.y].unsqueeze(0)
        preds = model(ins.unsqueeze(0)).squeeze()
        # print(outs, preds)
        errors.append(evaluate_results(outs.squeeze().cpu(), preds.cpu(), False))
        new_data = deepcopy(labels[test_loader.dataset.dataset.slice_outputs(start+i)])
        new_data[:,test_loader.dataset.dataset.y] = preds

        labs = torch.cat((labs, new_data))
    return np.array(errors)

forecast(model, labels, 30, 6)

array([[0.9975586 , 1.0931053 ],
       [1.4265137 , 0.2604561 ],
       [1.1947021 , 3.9334984 ],
       [1.3208008 , 0.10424423],
       [1.5064697 , 0.34140778],
       [1.3782959 , 3.3658638 ]], dtype=float32)

In [9]:
from tqdm import tqdm

ws = test_loader.dataset.dataset.num_inputs
print(list(range(0, test_loader.dataset.dataset.interval*ws, test_loader.dataset.dataset.interval)))
errors = []
model = model.to("cuda")
for seq in tqdm(test_loader):
    labels = seq[0].to("cuda")
    for start in range(len(labels) - (ws*2*test_loader.dataset.dataset.interval)):
        errors.append(forecast(model, labels, start, 12))
errors = np.array(errors)

print(np.mean(errors, axis=0))



[0, 3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33]


100%|██████████| 157/157 [03:47<00:00,  1.45s/it]

[[ 1.3393258  3.156114 ]
 [ 2.2639089  4.3077245]
 [ 3.3337069  5.652008 ]
 [ 4.4285984  7.0158324]
 [ 5.5052767  8.462587 ]
 [ 6.50806    9.819918 ]
 [ 7.4444737 11.023597 ]
 [ 8.312282  12.152789 ]
 [ 9.106851  13.172267 ]
 [ 9.832729  14.046605 ]
 [10.491654  14.808446 ]
 [11.087066  15.448015 ]]


In [28]:
print(np.std(errors, axis=0))

[[ 1.2280691  3.7533581]
 [ 1.8718818  4.542712 ]
 [ 2.7222672  5.52035  ]
 [ 3.661643   6.5948963]
 [ 4.5781794  7.602788 ]
 [ 5.4922676  8.527373 ]
 [ 6.4136057  9.336712 ]
 [ 7.3256984 10.119459 ]
 [ 8.204576  10.869802 ]
 [ 9.049443  11.562635 ]
 [ 9.864355  12.135972 ]
 [10.636837  12.7766075]]
